In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import random
from HAN import HAN

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
year='2013'
word2vec_size=50
cut_long_sentence=200
word2vec_model = KeyedVectors.load_word2vec_format('data/yelp'+year+'/yelp_'+year+'_'+str(word2vec_size)+'.vector')

2018-06-08 15:29:17,098 : INFO : loading projection weights from data/yelp2013/yelp_2013_50.vector
2018-06-08 15:29:19,489 : INFO : loaded (43531, 50) matrix from data/yelp2013/yelp_2013_50.vector


In [3]:
f_vocab=open('data/yelp'+year+'/yelp_'+year+'_'+str(word2vec_size)+'.vocab')
vocab_set=[]
for line in f_vocab:
    vocab_set.append(line.strip().split(' ')[0])
vocab_set=set(vocab_set)
f_vocab.close()
print('vocabulary size =',len(vocab_set))

vocabulary size = 43531


In [4]:
num_classes=5

In [5]:
tf.reset_default_graph()

X_sentence=tf.placeholder(dtype=tf.float32,shape=[None,None,None,word2vec_size]) # shape=(batch_size,doc_size,sen_size,vocab)
y=tf.placeholder(dtype=tf.int32,shape=[None])
sentence_length=tf.placeholder(dtype=tf.int32,shape=[None])  # it should have shape=(batch_size*document_size,)
document_length=tf.placeholder(dtype=tf.int32,shape=[None])  # it should have shape=(batch_size,)
is_training=tf.placeholder(dtype=tf.bool,shape=None)
dropout=tf.placeholder(dtype=tf.float32,shape=None)


model=HAN(num_classes,word2vec_size,lamb=0.0,disTill=2.0)
loss,predict,accuracy,attention_low,attention_high=model(X_sentence,y,sentence_length,document_length,dropout,is_training)

In [6]:
def make_input(doc):
    this_doc=[]
    sents=doc.split('\t')
    for s in sents:
        this_sentence=[]
        words=s.split(' ')
        for w in words:
            if w in vocab_set:
                this_sentence.append(w)
            else:
                this_sentence.append('<UNKNOWN>')
        this_doc.append(this_sentence)
        document_sizes=np.array([len(this_doc)])
        sentence_sizes=np.array([[len(t) for t in this_doc]])
        document_size=np.max(document_sizes)
        sentence_size=np.max(sentence_sizes)
        
        output=np.zeros((1,document_size,sentence_size,word2vec_size))
        
        for (id_s,s) in enumerate(this_doc):
            for (id_w,w) in enumerate(s):
                if id_w>=cut_long_sentence:  # cut too long sentences
                    break
                output[0,id_s,id_w,:]=word2vec_model[w]
        
        
    return this_doc,document_sizes,sentence_sizes,output

In [12]:
# f_dev=open('data/yelp'+year+'/yelp-'+year+'-dev-nonvocab.txt')  # randomly select some cases from dev set
# cases=[]
# for line in f_dev:
#     cases.append(line.strip().split('\t\t'))
# random.shuffle(cases)
# cases=cases[:10]

f_dev=open('data/yelp'+year+'/test.txt')
cases=[]
for line in f_dev:
    cases.append(line.strip().split('\t\t'))
random.shuffle(cases)
cases=cases[:1]

In [13]:
saver=tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "parameters/HAN.ckpt")
    cor=0
    for id,(label,sent) in enumerate(cases):
        this_doc,document_sizes,sentence_sizes,output=make_input(sent)
        
        slen=output.shape[2]
        
        label=np.array([int(label)-1])
        feed_dict={X_sentence:output,y:label,sentence_length:sentence_sizes.reshape(-1,),
                   document_length:document_sizes,is_training:False,dropout:0.0}
        pre,al,ah=sess.run([predict,attention_low,attention_high],feed_dict=feed_dict)
        print('Case #'+str(id+1)+':')
        print('Original sentence:')
        print(', '.join(sent.split('\t')))
        print()
        print('Exact label: %d, predict label: %d' % (label[0]+1,pre[0]+1))
        print()
        print('Sentence attention:')
        for (id_s,s) in enumerate(this_doc):
            print('%.2f%%' % (ah[0,id_s]*100.0),' '.join(s))
        print()
        print('Word attention:')
        for (id_s,s) in enumerate(this_doc):
            print(' '.join([str(('%.2f%%' % (al[id_s,id_w]*100.0),w)) for (id_w,w) in enumerate(s)]))
#             print(' '.join([str(('%.2f%%' % (al[id_s,id_w]*100.0),s[id_w] if id_w<len(s) else '_')) for id_w in range(slen)]))
        print()
        print()

INFO:tensorflow:Restoring parameters from parameters/HAN.ckpt


2018-06-08 15:30:42,010 : INFO : Restoring parameters from parameters/HAN.ckpt


Case #1:
Original sentence:
a hidden gem, service is awesome, prices are reasonable, food is very good, we came in with a large group of motorcycles, they greeted us and made room for us right away, i felt like they were sincerely glad to have our business and i will be back for sure

Exact label: 4, predict label: 5

Sentence attention:
0.38% a hidden gem
69.98% service is awesome
10.55% prices are reasonable
18.96% food is very good
0.07% we came in with a large group of motorcycles
0.06% they greeted us and made room for us right away
0.00% i felt like they were sincerely glad to have our business and i will be back for sure

Word attention:
('0.16%', 'a') ('78.37%', 'hidden') ('21.47%', 'gem')
('0.00%', 'service') ('0.03%', 'is') ('99.97%', 'awesome')
('0.00%', 'prices') ('0.00%', 'are') ('100.00%', 'reasonable')
('0.00%', 'food') ('0.00%', 'is') ('0.00%', 'very') ('100.00%', 'good')
('0.00%', 'we') ('0.00%', 'came') ('0.00%', 'in') ('0.00%', 'with') ('0.78%', 'a') ('92.26%', 'larg